## Imports

In [ ]:
import logging
from datetime import datetime as DT
from datetime import timedelta as TD
from dateutil import parser

from rich import print
import pandas as pd
from pandas import Series
from pandas import DataFrame as DF
import plotly.express as px
from core.s3_utils import S3_Bucket
from core.config import *

## Setup

Lets extract some raw time series... 

In [ ]:
VIN = "VF1AG000864501346"

bucket = S3_Bucket()
def get_renault_raw_ts(vin:str) -> DF:
    return (
        bucket.read_parquet_df(f"raw_ts/renault/time_series/{vin}.parquet")
        .set_index("date", drop=False)
        .sort_index()
        .assign(soc=lambda raw_ts:raw_ts["charging.battery_level"] * 100)
    )
raw_tss:DF = get_renault_raw_ts(VIN)

## Time series visualization
Let's look for variables that we could use to estimate the soh.

In [ ]:
tss:DF = (
    raw_tss.rename(columns={"charging.battery_energy": "battery_energy", "charging.distance_to_complete_charge": "distance_to_complete_charge", "diagnostics.odometer": "odometer", "charging.battery_level": "battery_level","charging.estimated_range": "estimated_range"})
)


In [ ]:
COLS_TO_DISPALY = [
    # "battery_charge_type",
    "distance_to_complete_charge",
    # "plugged_in",
    # "status",
    # "dodometer",
    "battery_energy",
    "soc",
    "estimated_range"
]
# px.line(raw_ts, COLS_TO_DISPALY)
px.line(tss[COLS_TO_DISPALY], )

In [ ]:
COLS_TO_DISPLAY = [
    "distance_to_complete_charge",
    "battery_energy",
    "soc",
    "estimated_range"
]

fig = px.line(tss, x="odometer", y=COLS_TO_DISPLAY)

# Customize the layout
fig.update_layout(
    xaxis_title="Odometer (km)",
    yaxis_title="Values",
    title="Vehicle Data vs Odometer"
)

# Format x-axis ticks
fig.update_xaxes(tickformat=",d")

# Display the plot
fig.show()

# Soh based on battery_energy

In [ ]:
raw_filtered = tss.query("soc > 40")
raw_filtered = raw_filtered.query("battery_energy > 0")
soh = raw_filtered["battery_energy"] / raw_filtered["soc"] / 52 
px.line(soh)